In [1]:
from whispr import *
import os


In [27]:
# gene expression. 2.5ul rxns, 75% txtl
rxn_vol = 3
source_plate_type = '384PP_AQ_BP' 

os.getcwd()
folder = os.getcwd() + '/Experiments/230404-MC/'
sp_plasmids_file = folder + '040323_sp-exp.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0)
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

layout_genex_file = folder + '040323_plate-layout-exp.csv'
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = folder + '040323_mixing-table-exp.csv'
mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*rxn_vol, total_vol = rxn_vol, fill_with='Water', multiRpW = False)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol=folder+'040323_genex_protocol.xlsx')


Volumes used from each well for this protocol:
{'1_A1': 0.8, '1_A2': 0.075, '1_A3': 0.3, '1_A4': 0.225, '1_A5': 0.175, '1_A6': 0.175, '1_A7': 0.5}


In [28]:
protocol_genex_df = protocol_genex_df[0]

In [29]:
# everything gets txtl

# get destination wells
for layout_id, layout in enumerate([layout_genex]):    
    # reads plate layout and assigns wells to each reaction (rxn_loc; dict)
    labels = pd.unique(np.concatenate(layout.values))
    rxn_loc = {}
    for l in labels:
        if type(l) is str:
            index = layout[layout.isin([l])].stack().index
            rxn_loc[l] = []
            for i in index:
                rxn_loc[l].append(str(i[0]) + str(i[1]))


txtl_well = 'P24'
for label, dwell in rxn_loc.items():
    for d in dwell:
        print(label)
        txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
        row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                    'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
        protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)

protocol_genex_df.to_csv(folder+'protocol_genex.csv',index = False)
print('TXTL needed: '+str(protocol_genex_df[protocol_genex_df['Source Well'] == txtl_well]['Transfer Volume'].sum()/1000))
# carry out gene expression reaction

MalE1
MalE5
Pyc
TXTL needed: 6.75


In [30]:
protocol_genex_df

,Source Plate Name,Source Plate Type,Source Well,Destination Plate Name,Destination Well,Transfer Volume
0,Source[1],384PP_AQ_BP,A1,Destination[1],P1,400.0
1,Source[1],384PP_AQ_BP,A3,Destination[1],P1,100.0
2,Source[1],384PP_AQ_BP,A4,Destination[1],P1,75.0
3,Source[1],384PP_AQ_BP,A6,Destination[1],P1,175.0
4,Source[1],384PP_AQ_BP,A1,Destination[1],P2,75.0
5,Source[1],384PP_AQ_BP,A3,Destination[1],P2,100.0
6,Source[1],384PP_AQ_BP,A4,Destination[1],P2,75.0
7,Source[1],384PP_AQ_BP,A7,Destination[1],P2,500.0
8,Source[1],384PP_AQ_BP,A1,Destination[1],P3,325.0
9,Source[1],384PP_AQ_BP,A2,Destination[1],P3,75.0


In [37]:
# dilution. everything gets Tris, for 20x -> 57uL

##### doing it by hand if not too many reactions #####

# protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
# source_plate_type = '6RES_AQ_B' 
# tris_well = 'A1'
# tris_vol = 57*1000

# for dwell in protocol_genex_df['Destination Well'].unique():
#     row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
#                                 'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
#     protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

# protocol_dilution_df.to_csv('protocols/221024_arpae_dilutions.csv',index = False)

In [ ]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [58]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = folder + '040323_sp-biosyn.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0)
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir (actually same plate as Tris)
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
sp_genex = sp_from_layout(layout_genex, 60)

# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP2','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = folder + '040323_mixing-table-biosyn.csv'
mt_biosyn = pd.read_csv(mt_biosyn_file, index_col = 0, dtype = str).fillna(0)
mt_biosyn = mt_biosyn[~mt_biosyn.index.isna()]

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
layout_biosyn1_file = folder + '040323_plate-layout-biosyn.csv' 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
# filename = '221018_arpae_biosyn_neg.csv'
# layout_biosyn2_file = 'plate_layouts/'+filename 
# layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
# layouts = [layout_biosyn1,layout_biosyn2]

vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

protocol_biosyn_dfs = writeProtocol(sp_types, vol_table_df, layout_biosyn1,sps, update_source_vol= folder +'combined_sp_updated.xlsx')

#protocol_biosyn_df.to_csv('protocols/221018_arpae_biosyn.csv',index = False)

Volumes used from each well for this protocol:
{'1_A1': 10.8, '1_A2': 16.5, '1_A3': 8.25, '1_A4': 18.0, '1_A5': 40.0, '1_B5': 40.0, '1_C5': 10.0, '1_A6': 3.6, '1_A7': 14.4, '1_A8': 33.0, '1_A9': 27.0, '1_A10': 16.5, '1_A11': 36.95, '1_A12': 0.95, '1_A13': 0.95, '2_A2': 565.6, '3_P1': 22.5, '3_P2': 22.5, '3_P3': 22.5, '3_P4': 12.5}


In [57]:
protocol_biosyn_dfs[0].to_csv(folder +'protocols_biosyn_384.csv',index = False)
protocol_biosyn_dfs[1].to_csv(folder +'protocols_biosyn_6RES.csv',index = False)

In [53]:
vol_table_df

,Label,Biotin,ATP,AcCoA,Mg-ACT,HCT,Kan,Formate,Bicarbonate,KOH,NADH,Pyruvate,Malate,Serine,HEPES,MalE1,MalE5,Pyc,Std
0,ME1,0.0,0.5,0.25,0.5,2.5,0.1,0.4,1.0,0.75,0.5,1.0,0.0,0.0,12.5,2.5,0.0,0.0,0.0
1,ME5,0.0,0.5,0.25,0.5,2.5,0.1,0.4,1.0,0.75,0.5,1.0,0.0,0.0,12.5,0.0,2.5,0.0,0.0
2,ME1ATPDIM,0.0,0.25,0.25,0.5,2.5,0.1,0.4,1.0,0.75,0.5,1.0,0.0,0.0,12.75,2.5,0.0,0.0,0.0
3,ME5ATPDIM,0.0,0.25,0.25,0.5,2.5,0.1,0.4,1.0,0.75,0.5,1.0,0.0,0.0,12.75,0.0,2.5,0.0,0.0
4,ME1ATPNIL,0.0,0.0,0.25,0.5,2.5,0.1,0.4,1.0,0.75,0.5,1.0,0.0,0.0,13.0,2.5,0.0,0.0,0.0
5,ME5ATPNIL,0.0,0.0,0.25,0.5,2.5,0.1,0.4,1.0,0.75,0.5,1.0,0.0,0.0,13.0,0.0,2.5,0.0,0.0
6,ME1NADHDIM,0.0,0.5,0.25,0.5,2.5,0.1,0.4,1.0,0.75,0.25,1.0,0.0,0.0,12.75,2.5,0.0,0.0,0.0
7,ME5NADHDIM,0.0,0.5,0.25,0.5,2.5,0.1,0.4,1.0,0.75,0.25,1.0,0.0,0.0,12.75,0.0,2.5,0.0,0.0
8,ME1NADHNIL,0.0,0.5,0.25,0.5,2.5,0.1,0.4,1.0,0.75,0.0,1.0,0.0,0.0,13.0,2.5,0.0,0.0,0.0
9,ME5NADHNIL,0.0,0.5,0.25,0.5,2.5,0.1,0.4,1.0,0.75,0.0,1.0,0.0,0.0,13.0,0.0,2.5,0.0,0.0
